# Laboratorio: Predizione Indice di Borsa (bozza)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


## Estrazione Dati

I dati storici sull'andamento di S&P 500 e altri indici di borsa sono disponibili su diversi siti specializzati, ad es. [https://finance.yahoo.com/](Yahoo! Finance)

Il file `SP500-2001-2005.csv` contiene l'andamento di S&P 500 dall'inizio del 2001 alla fine del 2005

In [34]:
sp500 = pd.read_csv("SP500-2001-2005.csv", parse_dates=["Date"])

Per ottenere questo file da Yahoo! Finance abbiamo cercato il titolo "S&P 500" (simbolo GSPC), selezionato la sezione "Historical Data", filtrato sul periodo da "1/1/2001" a "12/31/2005" e cliccato su "Download Data" per scaricare il CSV

## Interpretazione dei Dati

In [35]:
sp500.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2001-01-02,1320.280029,1320.280029,1276.050049,1283.270020,1283.270020,1129400000
1,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000
2,2001-01-04,1347.560059,1350.239990,1329.140015,1333.339966,1333.339966,2131000000
3,2001-01-05,1333.339966,1334.770020,1294.949951,1298.349976,1298.349976,1430800000
4,2001-01-08,1298.349976,1298.349976,1276.290039,1295.859985,1295.859985,1115500000


La colonna `Date` indica la data a cui si riferisce ciascuna riga, per cui è opportuno impostarla come indice

In [36]:
sp500.set_index("Date", inplace=True)

- `Open` indica il valore del titolo all'apertura del mercato (tutti i valori sono in USD)
- `Close` indica il valore alla chiusura del mercato (l'apertura del giorno dopo può differire)
- `High` e `Low` indicano il valore massimo e minimo raggiunti dal titolo nell'arco della giornata
- `Adj Close` è una versione corretta del prezzo di chiusura, in questo caso è sempre uguale a `Close`
- `Volume` indica il numero totale di titoli scambiati durante quel giorno 

## Obiettivo: Prevedere Salita o Discesa del Valore

Dai valori $O$ di apertura (`Open`) e $C$ di chiusura (`Close`) di ogni giorno possiamo definire la variazione percentuale del valore come

$$ 100\cdot\frac{C-O}{O} $$

Aggiungiamo una colonna con questo valore al dataset

In [37]:
sp500["PercReturn"] = 100 * (sp500["Close"] - sp500["Open"]) / sp500["Open"]

Il nostro obiettivo è, all'inizio di ogni giorno, prevedere se tale valore sarà positivo ($C>O$, valore in salita) o negativo ($C<O$, valore in discesa)

## Estrazione delle Feature Predittive

Per effettuare la predizione, servono degli indicatori il cui valore è noto all'inizio di ogni giornata: possiamo usare i valori dei giorni precedenti

Il metodo `shift` crea una copia di una serie con i valori slittati in avanti di un numero di righe indicato: la usiamo per creare 5 serie con i valori della colonna `PercReturn` ritardati da 1 a 5 giorni

In [38]:
lagged_cols = {i: sp500["PercReturn"].shift(i) for i in range(1, 6)}

Creo un nuovo DataFrame `X` con questi valori

In [39]:
for i in range(1, 6):
    sp500["Lag{}".format(i)] = sp500["PercReturn"].shift(i)

In [40]:
X = pd.DataFrame({"Lag{}".format(i): ser for i, ser in lagged_cols.items()})

Aggiungiamo a `X` una colonna col volume del giorno precedente, diviso per un miliardo ($10^9$) per avere una scala simile alle altre colonne

In [41]:
X["VolumeLag1"] = sp500["Volume"].shift(1) / 1e9

In [42]:
X.head(7)

,Lag1,Lag2,Lag3,Lag4,Lag5,VolumeLag1
Date,,,,,,
2001-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,-2.803194,NaN,NaN,NaN,NaN,1.1294
2001-01-04,5.009861,-2.803194,NaN,NaN,NaN,1.8807
2001-01-05,-1.055247,5.009861,-2.803194,NaN,NaN,2.1310
2001-01-08,-2.624236,-1.055247,5.009861,-2.803194,NaN,1.4308
2001-01-09,-0.191781,-2.624236,-1.055247,5.009861,-2.803194,1.1155
2001-01-10,0.381219,-0.191781,-2.624236,-1.055247,5.009861,1.1913


Nelle prime righe del dataset rimangono valori mancanti (_NaN_, Not a Number): usiamo il metodo `dropna` per rimuoverle le righe in cui sono presenti

In [43]:
X.dropna(inplace=True)

In [44]:
X.head(7)

,Lag1,Lag2,Lag3,Lag4,Lag5,VolumeLag1
Date,,,,,,
2001-01-09,-0.191781,-2.624236,-1.055247,5.009861,-2.803194,1.1155
2001-01-10,0.381219,-0.191781,-2.624236,-1.055247,5.009861,1.1913
2001-01-11,0.958639,0.381219,-0.191781,-2.624236,-1.055247,1.2965
2001-01-12,1.031770,0.958639,0.381219,-0.191781,-2.624236,1.4112
2001-01-16,-0.623287,1.031770,0.958639,0.381219,-0.191781,1.2760
2001-01-17,0.631871,-0.623287,1.031770,0.958639,0.381219,1.2057
2001-01-18,0.212561,0.631871,-0.623287,1.031770,0.958639,1.3491


Questo costituisce il set di variabili che utilizzeremo per predire la variazione giornaliera

Chiamiamo `y` la serie di valori che dobbiamo predire: usiamo `reindex_like` per selezionare i valori di `PercReturn` corrispondenti alle sole righe di `X` rimaste

In [45]:
y = sp500["PercReturn"].reindex_like(X)

## Divisione in Training e Validation Set

Dividiamo i dati ottenuti in due insiemi
- un _training set_ per addestrare i modelli di predizione
- un _validation set_ per valutare la loro accuratezza nella predizione, in modo da selezionare quello migliore

Usiamo i dati dal 2001 al 2004 (80% circa) per l'addestramento e quelli del 2005 per la validazione: creiamo un array binario `is_train` che indichi quali righe andranno nel training set piuttosto che nel validation set...

In [51]:
is_train = X.index.year < 2005

...quindi creiamo copie di `X` e `y` con i dati suddivisi tra training e validation

In [52]:
X_train = X[is_train]
y_train = y[is_train]
X_val = X[~is_train]
y_val = y[~is_train]

## Regressione Lineare

Testiamo dapprima un semplice modello di regressione lineare per prevedere la variazione odierna `y` dagli indicatori `X`

Creiamo un modello con i parametri di default

In [55]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

Addestriamo il modello con i dati di addestramento

In [56]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Otteniamo dal modello predizioni relative al validation set

In [57]:
y_val_pred = model.predict(X_val)

Per misurare l'accuratezza del modello, vediamo in quanti casi il segno della variazione + o - è previsto correttamente

Estraiamo una serie con il segno corretto...

In [58]:
s_val = pd.Series(np.where(y_val >= 0, "Up", "Down"), index=y_val.index)

...e una con il segno predetto

In [60]:
s_val_pred = pd.Series(np.where(y_val_pred >= 0, "Up", "Down"), index=y_val.index)

In che percentuale di casi i segni combaciano?

In [61]:
np.mean(s_val == s_val_pred)

0.5317460317460317

## Regressione Logistica

La regressione logistica è usata per prevedere una classe piuttosto che un valore reale, la usiamo per prevedere direttamente il segno + o - della variazione

In [62]:
s_train = pd.Series(np.where(y_train >= 0, "Up", "Down"), index=y_train.index)

In [63]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [64]:
model.fit(X_train, s_train)

/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [65]:
model.score(X_val, s_val)

0.4642857142857143

In [66]:
X_train.shape, X_val.shape

((999, 6), (252, 6))

In [67]:
X_train = X_train[["Lag1", "Lag2"]]
X_val = X_val[["Lag1", "Lag2"]]

In [68]:
model = LogisticRegression()

In [69]:
model.fit(X_train, s_train)

/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [70]:
model.score(X_val, s_val)

0.5753968253968254

In [71]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()
model.fit(X_train, s_train)
model.score(X_val, s_val)

0.5753968253968254

In [72]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model = QuadraticDiscriminantAnalysis()
model.fit(X_train, s_train)
model.score(X_val, s_val)

0.5753968253968254

In [76]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
model = Pipeline([
    ("poly", PolynomialFeatures(degree=10)),
    ("logreg", LogisticRegression())
])
model.fit(X_train, s_train)
model.score(X_val, s_val)

/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5

In [88]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from sklearn.kernel_ridge import KernelRidge
split = PredefinedSplit(np.where(is_train, -1, 0))
scorer = make_scorer(lambda y, p: np.mean((y>=0) == (p>=0)), greater_is_better=True)

In [94]:
gs = GridSearchCV(KernelRidge(), param_grid=[
    {"alpha": np.logspace(-4, 4, 5), "kernel": ["linear"]},
    {"alpha": np.logspace(-4, 4, 5), "kernel": ["poly"], "degree": np.linspace(2, 5, 4)},
    {"alpha": np.logspace(-4, 4, 5), "kernel": ["rbf"], "gamma": np.logspace(-4, 4, 5)}
], cv=split, scoring=scorer)
gs.fit(X[["Lag1", "Lag2"]], y)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
       error_score='raise-deprecating',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
      kernel_params=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]), 'kernel': ['linear']}, {'alpha': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]), 'kernel': ['poly'], 'degree': array([2., 3., 4., 5.])}, {'alpha': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]), 'kernel': ['rbf'], 'gamma': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(<lambda>), verbose=0)

In [97]:
pd.DataFrame(gs.cv_results_).sort_values("rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_kernel,param_degree,param_gamma,params,split0_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,mean_train_score,std_train_score
35,0.043692,0.0,0.004665,0.0,1,rbf,NaN,0.0001,"{'alpha': 1.0, 'gamma': 0.0001, 'kernel': 'rbf'}",0.579365,0.579365,0.0,1,0.517518,0.517518,0.0
28,0.094649,0.0,0.011124,0.0,0.0001,rbf,NaN,100,"{'alpha': 0.0001, 'gamma': 100.0, 'kernel': 'r...",0.567460,0.567460,0.0,2,0.992993,0.992993,0.0
30,0.071652,0.0,0.009744,0.0,0.01,rbf,NaN,0.0001,"{'alpha': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'}",0.563492,0.563492,0.0,3,0.527528,0.527528,0.0
45,0.065446,0.0,0.008373,0.0,10000,rbf,NaN,0.0001,"{'alpha': 10000.0, 'gamma': 0.0001, 'kernel': ...",0.559524,0.559524,0.0,4,0.506507,0.506507,0.0
40,0.063823,0.0,0.012050,0.0,100,rbf,NaN,0.0001,"{'alpha': 100.0, 'gamma': 0.0001, 'kernel': 'r...",0.559524,0.559524,0.0,4,0.506507,0.506507,0.0
0,0.318040,0.0,0.001367,0.0,0.0001,linear,NaN,NaN,"{'alpha': 0.0001, 'kernel': 'linear'}",0.551587,0.551587,0.0,6,0.521522,0.521522,0.0
1,0.072132,0.0,0.002470,0.0,0.01,linear,NaN,NaN,"{'alpha': 0.01, 'kernel': 'linear'}",0.551587,0.551587,0.0,6,0.521522,0.521522,0.0
2,0.052242,0.0,0.002525,0.0,1,linear,NaN,NaN,"{'alpha': 1.0, 'kernel': 'linear'}",0.551587,0.551587,0.0,6,0.521522,0.521522,0.0
3,0.059884,0.0,0.001281,0.0,100,linear,NaN,NaN,"{'alpha': 100.0, 'kernel': 'linear'}",0.551587,0.551587,0.0,6,0.521522,0.521522,0.0
4,0.049757,0.0,0.002629,0.0,10000,linear,NaN,NaN,"{'alpha': 10000.0, 'kernel': 'linear'}",0.551587,0.551587,0.0,6,0.520521,0.520521,0.0


## Simulazione Trading

Quanto guadagnerebbe un trader che compra e vende titoli in base alle predizioni del modello?

Ipotizziamo un trader che all'inizio di ogni giornata compra o vende (allo scoperto) un singolo titolo al suo valore di apertura a seconda della predizione del modello, per poi rivenderlo o ricomprarlo al suo valore di chiusura

Siano $O,C,\hat{C}$ rispettivamente il valore di apertura, quello di chiusura reale e quello di chiusura predetto, il guadagno (o perdita se negativo) di ciascuna giornata sarà:

$$ R = \left\{\begin{array}{lr}C-O&:\hat{C}>O\\O-C&:\hat{C}<O\end{array}\right. $$

In forma di funzione abbiamo

In [112]:
def gain(real, pred):
    real_data = sp500.reindex(index=real.index)
    diff = real_data["Close"] - real_data["Open"]
    return np.where(pred >= 0, diff, -diff).sum()
gain_scorer = make_scorer(gain, greater_is_better=True)

In [113]:
gs = GridSearchCV(KernelRidge(), param_grid=[
    {"alpha": np.logspace(-4, 4, 5), "kernel": ["linear"]},
    {"alpha": np.logspace(-4, 4, 5), "kernel": ["poly"], "degree": np.linspace(2, 5, 4)},
    {"alpha": np.logspace(-4, 4, 5), "kernel": ["rbf"], "gamma": np.logspace(-4, 4, 5)}
], cv=split, scoring=gain_scorer)
gs.fit(X[["Lag1", "Lag2"]], y)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
       error_score='raise-deprecating',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
      kernel_params=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]), 'kernel': ['linear']}, {'alpha': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]), 'kernel': ['poly'], 'degree': array([2., 3., 4., 5.])}, {'alpha': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]), 'kernel': ['rbf'], 'gamma': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(gain), verbose=0)

In [114]:
pd.DataFrame(gs.cv_results_).sort_values("rank_test_score")

/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('std_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_kernel,param_degree,param_gamma,params,split0_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,mean_train_score,std_train_score
35,0.042487,0.0,0.004573,0.0,1,rbf,NaN,0.0001,"{'alpha': 1.0, 'gamma': 0.0001, 'kernel': 'rbf'}",238.659534,238.659534,0.000000e+00,1,-152.358272,-152.358272,0.0
30,0.045742,0.0,0.004804,0.0,0.01,rbf,NaN,0.0001,"{'alpha': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'}",224.860954,224.860954,0.000000e+00,2,262.960690,262.960690,0.0
37,0.051469,0.0,0.006099,0.0,1,rbf,NaN,1,"{'alpha': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}",220.480346,220.480346,0.000000e+00,3,2657.818228,2657.818228,0.0
21,0.037180,0.0,0.001889,0.0,10000,poly,2,NaN,"{'alpha': 10000.0, 'degree': 2.0, 'kernel': 'p...",213.620970,213.620970,0.000000e+00,4,340.941906,340.941906,0.0
41,0.099801,0.0,0.015269,0.0,100,rbf,NaN,0.01,"{'alpha': 100.0, 'gamma': 0.01, 'kernel': 'rbf'}",194.240596,194.240596,0.000000e+00,5,417.841312,417.841312,0.0
0,0.075425,0.0,0.003908,0.0,0.0001,linear,NaN,NaN,"{'alpha': 0.0001, 'kernel': 'linear'}",193.280634,193.280634,0.000000e+00,6,261.820194,261.820194,0.0
1,0.055296,0.0,0.003404,0.0,0.01,linear,NaN,NaN,"{'alpha': 0.01, 'kernel': 'linear'}",193.280634,193.280634,0.000000e+00,6,261.820194,261.820194,0.0
2,0.047877,0.0,0.005882,0.0,1,linear,NaN,NaN,"{'alpha': 1.0, 'kernel': 'linear'}",193.280634,193.280634,0.000000e+00,6,261.820194,261.820194,0.0
3,0.045004,0.0,0.001495,0.0,100,linear,NaN,NaN,"{'alpha': 100.0, 'kernel': 'linear'}",193.280634,193.280634,0.000000e+00,6,261.820194,261.820194,0.0
4,0.069625,0.0,0.001512,0.0,10000,linear,NaN,NaN,"{'alpha': 10000.0, 'kernel': 'linear'}",193.280634,193.280634,0.000000e+00,6,250.560184,250.560184,0.0


In [111]:
model = LinearRegression()
model.fit(X_train[["Lag1", "Lag2"]], y_train)
gain(y_val, model.predict(X_val[["Lag1", "Lag2"]]))

/home/pasolini/.virtualenvs/dia/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  


218.64074000000028